# Nested types

* List, List View
* Struct
* Map
* Union
* Dictionary encoded

In nested types we introduce the concept of **parent** and **child arrays**. They express relationships between physical value arrays in a nested type structure.

Nested data type structure depends on one or more other child data types.
For instance, List is a nested type (parent) that has one child.

## List

The list type enables values of the same type being stacked together in a sequence of values in each column slot. The layout is similar to binary or string type as it has offsets buffer to define where the sequence of values starts and ends with all the values of the column being stored consecutively in a values buffer.

![image info](./diagrams/var-list-diagram.svg)

In [55]:
import nanoarrow as na
import numpy as np
import pyarrow as pa

In [56]:
column_1 = pa.array([[12, -7, 25], None, [0, -127, 127, 50], []],
                    type=pa.list_(pa.int8()))
column_1

[
  [
    12,
    -7,
    25
  ],
  null,
  [
    0,
    -127,
    127,
    50
  ],
  []
]

When inspecting a list type column (and all nested data in general) using pyarrow, the `buffers()` method returns all buffers - of list array itself (validity bitmap buffer and offset buffer) and its child array (validity bitmap buffer and values buffer):

In [57]:
# Inspecting buffers using PyArrow and buffers() method
column_1.buffers()

[<pyarrow.Buffer address=0x4bdc4020740 size=1 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc4020800 size=20 is_cpu=True is_mutable=True>,
 None,
 <pyarrow.Buffer address=0x4bdc4020b40 size=7 is_cpu=True is_mutable=True>]

In [58]:
# Inspecting buffers using PyArrow and buffers() method and numpy

validity_bitmap_buffer = column_1.buffers()[0]
np.unpackbits(np.frombuffer(validity_bitmap_buffer, dtype="uint8"), bitorder="little")

array([1, 0, 1, 1, 0, 0, 0, 0], dtype=uint8)

In [59]:
offsets_buffer = column_1.buffers()[1]
np.frombuffer(offsets_buffer, dtype="int32")

array([0, 3, 3, 7, 7], dtype=int32)

In [60]:
values_validity_bitmap_buffer = column_1.buffers()[2]
values_validity_bitmap_buffer is None

True

In [61]:
values_buffer = column_1.buffers()[3]
np.frombuffer(values_buffer, dtype="int8")

array([  12,   -7,   25,    0, -127,  127,   50], dtype=int8)

In [62]:
# Inspecting buffers using nanoarrow

na_column4 = na.c_array(column_1)
na.c_array_view(na_column4)

<nanoarrow.c_lib.CArrayView>
- storage_type: 'list'
- length: 4
- offset: 0
- null_count: 1
- buffers[2]:
  - validity <bool[1 b] 10110000>
  - data_offset <int32[20 b] 0 3 3 7 7>
- dictionary: NULL
- children[1]:
  - <nanoarrow.c_lib.CArrayView>
    - storage_type: 'int8'
    - length: 7
    - offset: 0
    - null_count: 0
    - buffers[2]:
      - validity <bool[0 b] >
      - data <int8[7 b] 12 -7 25 0 -127 127 50>
    - dictionary: NULL
    - children[0]:

### Fixed size list

**Fixed size list** is a special case of variable-size list where each column slot contains a fixed size sequence meaning all lists are the same size and so the offset buffer is no longer needed.

![image info](./diagrams/fixed-list-diagram.svg)

In [63]:
column_2 = pa.array([[12, -7], None, [0, None]], type=pa.list_(pa.int16(), 2))
na.c_array_view(na.c_array(column_2))

<nanoarrow.c_lib.CArrayView>
- storage_type: 'fixed_size_list'
- length: 3
- offset: 0
- null_count: 1
- buffers[1]:
  - validity <bool[1 b] 10100000>
- dictionary: NULL
- children[1]:
  - <nanoarrow.c_lib.CArrayView>
    - storage_type: 'int16'
    - length: 6
    - offset: 0
    - null_count: 3
    - buffers[2]:
      - validity <bool[1 b] 11001000>
      - data <int16[12 b] 12 -7 0 0 0 0>
    - dictionary: NULL
    - children[0]:

### List and large list comparison

In a normal list with variable or fixed size the offsets are `int32` while in the **large** list the offsets are `int64`.

In [64]:
column_1 = pa.array([[12, -7, 25], None, [0, -127, 127, 50], []],
                    type=pa.list_(pa.int8()))
na.c_array_view(na.c_array(column_1))

<nanoarrow.c_lib.CArrayView>
- storage_type: 'list'
- length: 4
- offset: 0
- null_count: 1
- buffers[2]:
  - validity <bool[1 b] 10110000>
  - data_offset <int32[20 b] 0 3 3 7 7>
- dictionary: NULL
- children[1]:
  - <nanoarrow.c_lib.CArrayView>
    - storage_type: 'int8'
    - length: 7
    - offset: 0
    - null_count: 0
    - buffers[2]:
      - validity <bool[0 b] >
      - data <int8[7 b] 12 -7 25 0 -127 127 50>
    - dictionary: NULL
    - children[0]:

In [65]:
column_1_large = pa.array([[12, -7, 25], None, [0, -127, 127, 50], []],
                          type=pa.large_list(pa.int8()))
na.c_array_view(na.c_array(column_1_large))

<nanoarrow.c_lib.CArrayView>
- storage_type: 'large_list'
- length: 4
- offset: 0
- null_count: 1
- buffers[2]:
  - validity <bool[1 b] 10110000>
  - data_offset <int64[40 b] 0 3 3 7 7>
- dictionary: NULL
- children[1]:
  - <nanoarrow.c_lib.CArrayView>
    - storage_type: 'int8'
    - length: 7
    - offset: 0
    - null_count: 0
    - buffers[2]:
      - validity <bool[0 b] >
      - data <int8[7 b] 12 -7 25 0 -127 127 50>
    - dictionary: NULL
    - children[0]:

### List and large list view

List view type allows arrays to specify out-of-order offsets.

In [66]:
column_3 = pa.ListViewArray.from_arrays(offsets=[4, 7, 0, 0, 3],
                                        sizes=[3, 0, 4, 0, 2],
                                        values=[0, -127, 127, 50, 12, -7, 25],
                                        mask=pa.array([False, True, False, False, False]))

In [67]:
column_3.buffers()

[<pyarrow.Buffer address=0x4bdc4021380 size=1 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc4050a80 size=20 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc4050b00 size=20 is_cpu=True is_mutable=True>,
 None,
 <pyarrow.Buffer address=0x4bdc4021340 size=56 is_cpu=True is_mutable=True>]

In [68]:
column_3

[
  [
    12,
    -7,
    25
  ],
  null,
  [
    0,
    -127,
    127,
    50
  ],
  [],
  [
    50,
    12
  ]
]

In [69]:
values_buffer = column_3.buffers()[4]
np.frombuffer(values_buffer, dtype="int64")

array([   0, -127,  127,   50,   12,   -7,   25])

### Struct

A struct is a nested type parameterized by an ordered sequence of types.

* one child array for each field
* child arrays are independent and need not be adjacent to each other in memory

One can think of an individual struct field as a key-value pair where the key is the field name and the child array its values. The field (key) is saved in the schema and the values of a specific field (key) are saved in the child array.

![image info](./diagrams/struct-diagram.svg)

In [70]:
ty = pa.struct([pa.field('x', pa.string()),
                pa.field('y', pa.int8())])
column_4 = pa.array([{"x": "joe", "y": 1},
                     {"x": None, "y": 2}, None,
                     {"x": "mark", "y": 4},
                     {"x": "jane", "y": None}],
                    type=ty)
column_4

-- is_valid:
  [
    true,
    true,
    false,
    true,
    true
  ]
-- child 0 type: string
  [
    "joe",
    null,
    "",
    "mark",
    "jane"
  ]
-- child 1 type: int8
  [
    1,
    2,
    0,
    4,
    null
  ]

In [71]:
column_4.buffers()

[<pyarrow.Buffer address=0x4bdc40213c0 size=1 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc4021440 size=1 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc4021400 size=24 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc4021500 size=11 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc40214c0 size=1 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc4021480 size=5 is_cpu=True is_mutable=True>]

In [72]:
na.c_array_view(na.c_array(column_4))

<nanoarrow.c_lib.CArrayView>
- storage_type: 'struct'
- length: 5
- offset: 0
- null_count: 1
- buffers[1]:
  - validity <bool[1 b] 11011000>
- dictionary: NULL
- children[2]:
  - <nanoarrow.c_lib.CArrayView>
    - storage_type: 'string'
    - length: 5
    - offset: 0
    - null_count: 1
    - buffers[3]:
      - validity <bool[1 b] 10111000>
      - data_offset <int32[24 b] 0 3 3 3 7 11>
      - data <string[11 b] b'joemarkjane'>
    - dictionary: NULL
    - children[0]:
  - <nanoarrow.c_lib.CArrayView>
    - storage_type: 'int8'
    - length: 5
    - offset: 0
    - null_count: 1
    - buffers[2]:
      - validity <bool[1 b] 11110000>
      - data <int8[5 b] 1 2 0 4 0>
    - dictionary: NULL
    - children[0]:

### Map

Map type represents nested data where each value is a variable number of key-value pairs. Its physical representation is the same as a list of `{key, value}` structs.

The difference between a struct and a map type is that a struct holds the key in the schema therefore needs to be a string, the values are stored in values buffer, one per each field. There can be multiple keys and therefore multiple values buffers. The map comparing to the struct has one child array holding all different keys that need to be all same type but not necessarily string and second child array holding all the values which also need to be the same type.

Also, the map stores the struct in a list and needs an offset as the list is variable shape.

![image info](./diagrams/map-diagram.svg)

In [73]:
column_6_data = [{'Dark Knight': 10},
                 {'Dark Knight': 8, 'Meet the Parents': 4, 'Superman': 5},
                 None,
                 {'Meet the Parents': 10, 'Superman': None}]
column_6 = pa.array(column_6_data, type=pa.map_(pa.string(), pa.int32()))
column_6

[
  keys:
  [
    "Dark Knight"
  ]
  values:
  [
    10
  ],
  keys:
  [
    "Dark Knight",
    "Meet the Parents",
    "Superman"
  ]
  values:
  [
    8,
    4,
    5
  ],
  null,
  keys:
  [
    "Meet the Parents",
    "Superman"
  ]
  values:
  [
    10,
    null
  ]
]

In [74]:
column_6.buffers()

[<pyarrow.Buffer address=0x4bdc4021580 size=1 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc4021540 size=20 is_cpu=True is_mutable=True>,
 None,
 None,
 <pyarrow.Buffer address=0x4bdc4021600 size=28 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc4050c00 size=70 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc4021680 size=1 is_cpu=True is_mutable=True>,
 <pyarrow.Buffer address=0x4bdc4050b80 size=24 is_cpu=True is_mutable=True>]

In [75]:
na.c_array_view(na.c_array(column_6))

<nanoarrow.c_lib.CArrayView>
- storage_type: 'map'
- length: 4
- offset: 0
- null_count: 1
- buffers[2]:
  - validity <bool[1 b] 11010000>
  - data_offset <int32[20 b] 0 1 4 4 6>
- dictionary: NULL
- children[1]:
  - <nanoarrow.c_lib.CArrayView>
    - storage_type: 'struct'
    - length: 6
    - offset: 0
    - null_count: 0
    - buffers[1]:
      - validity <bool[0 b] >
    - dictionary: NULL
    - children[2]:
      - <nanoarrow.c_lib.CArrayView>
        - storage_type: 'string'
        - length: 6
        - offset: 0
        - null_count: 0
        - buffers[3]:
          - validity <bool[0 b] >
          - data_offset <int32[28 b] 0 11 22 38 46 62 70>
          - data <string[70 b] b'D...>
        - dictionary: NULL
        - children[0]:
      - <nanoarrow.c_lib.CArrayView>
        - storage_type: 'int32'
        - length: 6
        - offset: 0
        - null_count: 1
        - buffers[2]:
          - validity <bool[1 b] 11111000>
          - data <int32[24 b] 10 8 4 5 10 0>
    

### Comparing the conversion of struct and map to python objects

Struct converts to a list of dictionaries:

In [76]:
column_4.to_numpy(zero_copy_only=False)

array([{'x': 'joe', 'y': 1.0}, {'x': None, 'y': 2.0}, None,
       {'x': 'mark', 'y': 4.0}, {'x': 'jane', 'y': None}], dtype=object)

In [77]:
column_4.to_pylist()

[{'x': 'joe', 'y': 1},
 {'x': None, 'y': 2},
 None,
 {'x': 'mark', 'y': 4},
 {'x': 'jane', 'y': None}]

And map converts to a nested list of tuples by default:

In [78]:
column_6.to_numpy(zero_copy_only=False)

array([list([('Dark Knight', 10.0)]),
       list([('Dark Knight', 8.0), ('Meet the Parents', 4.0), ('Superman', 5.0)]),
       None, list([('Meet the Parents', 10.0), ('Superman', None)])],
      dtype=object)

Or to dictionary with the use of `maps_as_pydicts` keyword:

In [86]:
# maps_as_pydicts can be ‘lossy’ or ‘strict’
# This can change the ordering of (key, value) pairs, and will deduplicate multiple keys, resulting in a possible loss of data.

# ‘lossy’: warning printed when detected
# ‘strict’ exception being raised when deduplicate detected
column_6.to_pandas(zero_copy_only=False, maps_as_pydicts="lossy")

0                                {'Dark Knight': 10.0}
1    {'Dark Knight': 8.0, 'Meet the Parents': 4.0, ...
2                                                 None
3         {'Meet the Parents': 10.0, 'Superman': None}
dtype: object

In [79]:
column_6.to_pylist()

[[('Dark Knight', 10)],
 [('Dark Knight', 8), ('Meet the Parents', 4), ('Superman', 5)],
 None,
 [('Meet the Parents', 10), ('Superman', None)]]